In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
print("PyTorch Version: ",torch.__version__)

PyTorch Version:  1.5.1


In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,20,5,1) #一共20个5*5的卷积核，生成20个特征图组
        self.conv2 = nn.Conv2d(20,50,5,1) # 输入时20个特征图组，50个5*5的卷积核
        self.fc1 = nn.Linear(4*4*50,500) #
        self.fc2 = nn.Linear(500,10) #十分类
    
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1,4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x,dim=1)

        

In [9]:
mnist_data = datasets.MNIST('./mnist_data',train = True,download = True
                                                           ,transform = transforms.Compose([
                                                             transforms.ToTensor(),
                                                         ]))

In [14]:
import numpy as np
mnist_data[20][0]

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [15]:
def train(model,device,train_loader,optimizer,epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target = data.to(device) , target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output,target)
        loss.backward()
        optimizer.step()
        if batch_idx %100 == 0:
            print("Train Epoch: {} [{}/{} ({:0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 
                100. * batch_idx / len(train_loader), loss.item()
            ))

In [18]:
def test(model,device,test_loader):
    model.eval()
    test_loss = 0
    corr = 0
    with torch.no_grad():
        for data,target in test_loader:
            data,target = data.to(device) , target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output,target,reduction = 'sum').item() #loss的求和
            pred = output.argmax(dim=1,keepdim = True) #10类中拿出概率最大的
            corr += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, corr, len(test_loader.dataset),
        100. * corr / len(test_loader.dataset)))

In [20]:
torch.manual_seed(0)

device = "cpu"
batch_size = test_batch_size = 32
kwargs = {} #没有使用gpu num_workers pin_memory都不设置了
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data',train = True,download = True
                                                           ,transform = transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307,),(0.3081,))
                                                         ])),batch_size=test_batch_size,shuffle=True,**kwargs)

test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data',train = False
                                                           ,transform = transforms.Compose([
                                                             transforms.ToTensor(),
                                                             transforms.Normalize((0.1307,),(0.3081,))
                                                         ])),batch_size=test_batch_size,shuffle=True,**kwargs)


lr =0.01
momentum = 0.9
model = Net().to(device)
optimizer = optim.SGD(model.parameters(),lr=lr,momentum=momentum)

epochs = 3
for epoch in range(1,epochs):
    train(model,device,train_loader,optimizer,epoch)
    test(model,device,test_loader)
    
save_model = True

if(save_model):
    torch.save(model.state_dict(),"mnist_cnn_simple_model.pt")
                                           


Train Epoch: 1 [0/60000 (0.000000%)]	Loss: 2.309706
Train Epoch: 1 [3200/60000 (5.333333%)]	Loss: 0.347881
Train Epoch: 1 [6400/60000 (10.666667%)]	Loss: 0.127570
Train Epoch: 1 [9600/60000 (16.000000%)]	Loss: 0.049735
Train Epoch: 1 [12800/60000 (21.333333%)]	Loss: 0.029335
Train Epoch: 1 [16000/60000 (26.666667%)]	Loss: 0.114104
Train Epoch: 1 [19200/60000 (32.000000%)]	Loss: 0.353635
Train Epoch: 1 [22400/60000 (37.333333%)]	Loss: 0.050342
Train Epoch: 1 [25600/60000 (42.666667%)]	Loss: 0.136899
Train Epoch: 1 [28800/60000 (48.000000%)]	Loss: 0.052979
Train Epoch: 1 [32000/60000 (53.333333%)]	Loss: 0.241112
Train Epoch: 1 [35200/60000 (58.666667%)]	Loss: 0.006182
Train Epoch: 1 [38400/60000 (64.000000%)]	Loss: 0.050151
Train Epoch: 1 [41600/60000 (69.333333%)]	Loss: 0.022593
Train Epoch: 1 [44800/60000 (74.666667%)]	Loss: 0.005063
Train Epoch: 1 [48000/60000 (80.000000%)]	Loss: 0.009157
Train Epoch: 1 [51200/60000 (85.333333%)]	Loss: 0.245035
Train Epoch: 1 [54400/60000 (90.666667%)

TypeError: 'DataLoader' object is not subscriptable